In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import talib  
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

tickers = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'SPY']#FAANG portfolio with SPY for market proxy

In [ ]:
class DataCollector:
    def __init__(self, tickers, start_date='2015-01-01', end_date='2024-12-31'):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.data = {}  # Will store raw data for each ticker
        
    def download_data(self):
        for ticker in self.tickers:
            try:
                # Download data for this ticker
                df = yf.download(
                    ticker, 
                    start=self.start_date,
                    end=self.end_date,
                    progress=False,  # Don't show progress bar
                    auto_adjust=True  # Use adjusted prices automatically
                )
                df.columns = df.columns.str.lower()
                df = df[['open', 'high', 'low', 'close', 'volume']]
                
                # Add ticker symbol as a column
                df['ticker'] = ticker
                
                # Store in dictionary
                self.data[ticker] = df
                
            except Exception as e:
                print("failure")
    
    def create_panel_data(self):
        # Create empty list to store DataFrames
        dfs = []
        
        for ticker, df in self.data.items():
            
            df = df.reset_index()
            df['ticker'] = ticker
            df.set_index(['ticker', 'Date'], inplace=True)
            dfs.append(df)
        
        # Combine all DataFrames
        panel_data = pd.concat(dfs, axis=0)
        
        # Sort by ticker and date for faster operations
        panel_data = panel_data.sort_index()
        
        return panel_data